<a href="https://colab.research.google.com/github/YinGuoX/Deep_Learning_Pytorch_WithDeeplizard/blob/master/34_PyTorch_DataLoader_Num_workers_Deep_Learning_Speed_Limit_Increase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PyTorch DataLoader num_workers Test - Speed Things Up
在这节中，我们将看到如何利用PyTorch DataLoader类的多进程功能来加速神经网络训练过程。

## 1.加快训练过程

为了加快训练过程，我们将使用DataLoader类的num_workers可选属性。

num_workers属性告诉数据加载器实例要用于数据加载的子进程数。 默认情况下，num_workers值设置为零，零值告诉加载程序将数据加载到主进程内部。

这意味着培训过程将在主要过程中按顺序进行。 在训练过程中使用了一个批次并且需要另一个批次之后，我们才从磁盘读取了批次数据。

现在，如果我们有一个工作进程，那么我们可以利用我们的机器具有多个内核的事实。 这意味着在主流程准备好另一个批次时，已经可以加载下一个批次并准备好运行。 这就是速度的来源。 批处理使用其他工作进程加载，并在内存中排队。

### num_workers属性的最佳值
---
自然的问题是，我们应该添加多少工作进程？这里有很多因素会影响最佳数字，所以最好的办法就是测试。


## 2.测试num_workers属性的值

要设置此测试，我们将创建一个num_workers值列表以进行尝试。 我们将尝试以下值：
* 0（默认）
* 1
* 2
* 4
* 8
* 16

对于这些值中的每一个，我们将通过尝试以下值来更改批处理大小：

* 100
* 1000
* 10000

对于学习率，对于所有运行，我们将其保持在.01的恒定值。

关于设置的最后一件事是，对于每个运行，我们只执行一个epoch。

好吧，让我们看看我们得到了什么。

### 不同的num_workers值的结果
---
好吧，我们可以在下面看到我们得到的结果。 我们总共完成了18次运行。 我们有三批不同的批处理大小，并且在每组中，我们都更改了工作进程的数量。

In [ ]:
# 实际的源码
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter

In [ ]:
class Network(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(in_channels=1,out_channels=6,kernel_size=5)
    self.conv2 = nn.Conv2d(in_channels=6,out_channels=12,kernel_size=5)

    self.fc1 = nn.Linear(in_features=12*4*4,out_features=120)
    self.fc2 = nn.Linear(in_features=120,out_features=60)
    self.out = nn.Linear(in_features=60,out_features=10)

  
  def forward(self,t):
    t= t

    t = self.conv1(t)
    t = F.relu(t)
    t = F.max_pool2d(t,kernel_size=2,stride=2)

    t = self.conv2(t)
    t = F.relu(t)
    t = F.max_pool2d(t,kernel_size=2,stride=2)

    t = t.reshape(-1,12*4*4)
    t = self.fc1(t)
    t = F.relu(t)

    t = self.fc2(t)
    t = F.relu(t)

    t = self.out(t)

    return t;


In [ ]:
train_set = torchvision.datasets.FashionMNIST(root='./data',train=True,
                                download=True,
                                transform=transforms.Compose([
        transforms.ToTensor()
    ]))


Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw




Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw




Extracting ./data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/raw




Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw

Processing...
Done!


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [ ]:
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from IPython.display import display, clear_output
import pandas as pd
import time
import json

from itertools import product
from collections import namedtuple
from collections import OrderedDict

In [ ]:
class RunBuilder():
    @staticmethod
    def get_runs(params):

        Run = namedtuple('Run', params.keys())

        runs = []
        for v in product(*params.values()):
          runs.append(Run(*v))
        return runs

In [ ]:
params = OrderedDict(
    lr = [0.01],
    batch_size = [100,1000,10000],
    num_workers=[0,1,2,4,8,16]
)

In [ ]:
for run in RunBuilder.get_runs(params):
  print(run.lr,run.batch_size,run.num_workers)

0.01 100 0
0.01 100 1
0.01 100 2
0.01 100 4
0.01 100 8
0.01 100 16
0.01 1000 0
0.01 1000 1
0.01 1000 2
0.01 1000 4
0.01 1000 8
0.01 1000 16
0.01 10000 0
0.01 10000 1
0.01 10000 2
0.01 10000 4
0.01 10000 8
0.01 10000 16


In [ ]:
# 理解上述思想，构建RunManager()类
class RunManager():
  def __init__(self):
    self.epoch_count = 0
    self.epoch_loss = 0
    self.epoch_num_correct = 0
    self.epoch_start_time = None
    self.epoch_num_workers=0


    self.run_params = None
    self.run_count = 0
    self.run_data = []
    self.run_start_time = None

    self.network = None
    self.loader = None
    self.tb = None

  
  def begin_run(self,run,network,loader):
    self.run_start_time = time.time()

    self.run_params = run
    self.run_count += 1

    self.network = network
    self.loader = loader
    self.tb = SummaryWriter(comment=f'-{run}')

    images, labels = next(iter(self.loader))
    grid = torchvision.utils.make_grid(images)

    self.tb.add_image('images', grid)
    self.tb.add_graph(self.network, images)

  def end_run(self):
    self.tb.close()
    self.epoch_count = 0

  def begin_epoch(self):
    self.epoch_start_time = time.time()
    self.epoch_count += 1
    self.epoch_loss = 0
    self.epoch_num_correct = 0


  def end_epoch(self):

    epoch_duration = time.time() - self.epoch_start_time
    run_duration = time.time() - self.run_start_time

    loss = self.epoch_loss / len(self.loader.dataset)
    accuracy = self.epoch_num_correct / len(self.loader.dataset)

    self.tb.add_scalar('Loss', loss, self.epoch_count)
    self.tb.add_scalar('Accuracy', accuracy, self.epoch_count)

    for name, param in self.network.named_parameters():
        self.tb.add_histogram(name, param, self.epoch_count)
        self.tb.add_histogram(f'{name}.grad', param.grad, self.epoch_count)

    results = OrderedDict()
    results["run"] = self.run_count
    results["epoch"] = self.epoch_count
    results['loss'] = loss
    results["accuracy"] = accuracy
    results['epoch duration'] = epoch_duration
    results['run duration'] = run_duration
    results['num_workers'] = self.epoch_num_workers
    for k,v in self.run_params._asdict().items(): results[k] = v
    self.run_data.append(results)

    df = pd.DataFrame.from_dict(self.run_data, orient='columns')
    
    clear_output(wait=True)
    
    display(df)

  def get_num_workers(self,num_workers):
    self.epoch_num_workers = num_workers


  def track_loss(self, loss, batch):
    self.epoch_loss += loss.item() * batch[0].shape[0]

  def track_num_correct(self, preds, labels):
    self.epoch_num_correct += self.get_num_correct(preds, labels)

  def get_num_correct(self, preds, labels):
    return preds.argmax(dim=1).eq(labels).sum().item()

  def save(self, fileName):

    pd.DataFrame.from_dict(
        self.run_data, orient='columns'
    ).to_csv(f'{fileName}.csv')

    with open(f'{fileName}.json', 'w', encoding='utf-8') as f:
        json.dump(self.run_data, f, ensure_ascii=False, indent=4)

In [ ]:
runManger = RunManager()
for run in RunBuilder.get_runs(params):
  print(run)
  network = Network()
  train_loader = DataLoader(
      train_set,
      batch_size =run.batch_size,
      num_workers=run.num_workers
  )
  optimizer = optim.Adam(network.parameters(),lr=run.lr)
  runManger.begin_run(run,network,train_loader)
  for epoch in range(1):
    runManger.begin_epoch()
    runManger.get_num_workers(run.num_workers)
    
    for batch in train_loader:
      images,labels = batch
      
      preds = network(images)

      loss = F.cross_entropy(preds,labels)

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      runManger.track_loss(loss,batch)
      runManger.track_num_correct(preds,labels)
    runManger.end_epoch()
  runManger.end_run()








,run,epoch,loss,accuracy,epoch duration,run duration,num_workers,lr,batch_size
0,1,1,0.519225,0.807800,12.689024,12.826912,0,0.01,100
1,2,1,0.577668,0.782917,11.841272,12.023044,1,0.01,100
2,3,1,0.528843,0.805400,12.099635,12.300997,2,0.01,100
3,4,1,0.542476,0.794683,12.235800,12.559361,4,0.01,100
4,5,1,0.618209,0.763200,12.305142,12.738291,8,0.01,100
5,6,1,0.549799,0.790217,12.499185,13.239710,16,0.01,100
6,7,1,0.986814,0.622767,11.683292,12.577870,0,0.01,1000
7,8,1,1.002655,0.616733,9.673973,10.676051,1,0.01,1000
8,9,1,0.932665,0.644650,9.816570,10.965652,2,0.01,1000
9,10,1,1.025632,0.610750,9.949142,11.302649,4,0.01,1000


从这些结果中得出的主要结论是，在所有三个批处理量中，除了主要流程外还具有单个工作流程可以使速度提高约20％。

* num_workers=1可以提高

此外，在第一个工作进程之后添加其他工作进程并没有真正显示出任何进一步的改进

## 3.解释结果

我们在添加一个worker进程后看到的20%的速度是有意义的，因为主进程要做的工作更少。

当主进程忙于执行正向传播和反向传播时，工作进程正在加载下一批。当主进程准备好进行另一批处理时，工作进程已经在内存中排队。

因此，主进程不必从磁盘读取数据。相反，数据已经在内存中了，这给了我们20%的速度。

现在，为什么我们在增加更多的工作进程之后没有看到更多的提速呢？

### 用更多的线程让它走得更快？
---
如果一个线程足以让队列中充满主进程的数据，那么向队列中添加更多的批数据将不会起任何作用。这就是我认为我们在这里看到的。

仅仅因为我们向队列中添加了更多批，并不意味着批处理的速度更快。因此，我们受到向前和向后传播给定批所需时间的限制。

我们甚至可以看到，当我们有16个线程时时，情况开始陷入困境。

希望这能帮助你加快训练速度！